In [10]:
import pandas as pd
from ollama import Client
from tqdm import tqdm
import time

In [4]:
# === Setup ===
ollama_client = Client()
OLLAMA_MODEL = "llama3.2:latest"  # Change to your preferred local model

In [8]:
USE_ORIGINAL_TABLES = False       # Set to False to skip original tables and use only candidate texts

# === CONFIGURATION ===
OLLAMA_MODEL = "llama3.2:latest"  # Change to any model you pulled with `ollama pull`
TABLE1_ID_COL = "custom_id"   # Replace with your actual ID column for table1
TABLE2_ID_COL = "custom_id"   # Replace with your actual ID column for table2
LEFT_ID_COL = "left_id"   # Column in candidate set that maps to table1
RIGHT_ID_COL = "right_id" # Column in candidate set that maps to table2

candidates = pd.read_csv("entity_matches.csv")
# === Load data ===
if USE_ORIGINAL_TABLES:
    table1 = pd.read_csv("walmart_amazon/tableA.csv")        # replace with your file
    table2 = pd.read_csv("walmart_amazon/tableB.csv")      # replace with your file

In [12]:

# === Ollama client ===
ollama_client = Client()

# === Row lookup ===
def get_row_by_id(df, id_col, id_val):
    return df[df[id_col] == id_val].iloc[0].to_dict()

# === Format prompt ===
def format_prompt(rowA, rowB):
    def format_row(row):
        return "\n".join([f"{k}: {str(v)}" for k, v in row.items() if pd.notnull(v) and str(v).strip() != ""])
    
    return f"""
Compare the following two products and determine if they refer to the same real-world item. Answer "Yes" if they match, otherwise "No".

Product A:
{format_row(rowA)}

Product B:
{format_row(rowB)}

Do these products refer to the same item?
""".strip()

# === Format prompt from candidate set directly ===
def format_prompt_from_candidate(row):
    return f"""
Do the two product descriptions refer to the same real-world product? Answer "Yes" if they match, otherwise "No".

Product A:
{row['table1_text']}

Product B:
{row['table2_text']}

Do these products refer to the same item?
""".strip()

# === Query LLM ===
def query_llm(prompt):
    response = ollama_client.chat(model=OLLAMA_MODEL, messages=[
        {"role": "user", "content": prompt}
    ])
    answer = response['message']['content'].strip().lower()
    return answer.startswith("yes")

# === Run matching ===
results = []
total_start = time.time()

for _, row in tqdm(candidates.iterrows(), total=len(candidates)):
    try:
        start_time = time.time()
        
        if USE_ORIGINAL_TABLES:
            rowA = get_row_by_id(table1, TABLE1_ID_COL, row[LEFT_ID_COL])
            rowB = get_row_by_id(table2, TABLE2_ID_COL, row[RIGHT_ID_COL])
            prompt = format_prompt(rowA, rowB)
        else:
            prompt = format_prompt_from_candidate(row)

        is_match = query_llm(prompt)
        elapsed = time.time() - start_time

        match_result = {
            LEFT_ID_COL: row[LEFT_ID_COL],
            RIGHT_ID_COL: row[RIGHT_ID_COL],
            "is_match": is_match,
            "time_sec": round(elapsed, 2)
        }
        results.append(match_result)

        print(f"Matched ({row[LEFT_ID_COL]}, {row[RIGHT_ID_COL]}) -> {'✅ Match' if is_match else '❌ No Match'} [Time: {elapsed:.2f}s]")

    except Exception as e:
        print(f"Error processing pair ({row[LEFT_ID_COL]}, {row[RIGHT_ID_COL]}): {e}")

# === Save results ===
output_df = pd.DataFrame(results)
output_df.to_csv("llm_matching_results.csv", index=False)

total_elapsed = time.time() - total_start
print(f"✅ Matching complete. Saved to llm_matching_results.csv.")
print(f"⏱️ Total matching time: {total_elapsed:.2f} seconds.")

  0%|                                                        | 1/25540 [00:02<15:15:24,  2.15s/it]

Matched (1, 2990) -> ❌ No Match [Time: 2.14s]


  0%|                                                        | 2/25540 [00:03<10:56:44,  1.54s/it]

Matched (1, 21844) -> ❌ No Match [Time: 1.12s]


  0%|                                                         | 3/25540 [00:04<8:31:05,  1.20s/it]

...

Matched (8, 9285) -> ❌ No Match [Time: 4.14s]


  0%|▏                                                      | 73/25540 [02:17<18:06:51,  2.56s/it]

Matched (8, 13269) -> ❌ No Match [Time: 1.93s]


  0%|▏                                                      | 73/25540 [02:18<13:24:53,  1.90s/it]


KeyboardInterrupt: 